In [1]:
import cell2fate as c2f
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import os

Global seed set to 0


In [2]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scvelo as scv
sns.reset_defaults()
sns.reset_orig()
scv.settings.set_figure_params('scvelo', dpi_save=400, dpi=80, transparent=True, fontsize=20, color_map='viridis')
import os
import anndata as ad
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
import anndata as ad


## Import cell cycle dataset and train the model

In [3]:
adata = sc.read("/gpfs/home/jw7078/cell_cycle_processed.h5ad")


In [4]:
adata

AnnData object with n_obs × n_vars = 1146 × 395
    obs: 'phase', 'fucci_time', 'pseudo_clusters', 'pseudo_clusters_equal_size', 'pseudo_clusters_equal_size_num', 'cell_cycle_rad', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'n_counts'
    var: 'gene_count_corr', 'means', 'dispersions', 'dispersions_norm', 'highly_variable', 'velocity_gamma', 'velocity_qreg_ratio', 'velocity_r2', 'velocity_genes'
    uns: 'neighbors', 'pca', 'umap', 'velocity_params'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'Ms', 'Mu', 'spliced', 'spliced_raw', 'unspliced', 'unspliced_raw', 'velocity'
    obsp: 'connectivities', 'distances'

In [5]:
clusters_to_remove = []
adata.obs["clusters"] = "0"

adata =  c2f.utils.get_training_data(adata, cells_per_cluster = 10**5, cluster_column = 'clusters',
                                    remove_clusters = clusters_to_remove,
                                min_shared_counts = 0, n_var_genes= 2000)

Keeping at most 100000 cells per cluster
Skip filtering by dispersion since number of variables are less than `n_top_genes`.


In [6]:
c2f.Cell2fate_DynamicalModel.setup_anndata(adata, spliced_label='spliced_raw', unspliced_label='unspliced_raw')


No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [7]:
n_modules = c2f.utils.get_max_modules(adata)


Leiden clustering ...
Number of Leiden Clusters: 5
Maximal Number of Modules: 5


In [8]:
mod = c2f.Cell2fate_DynamicalModel(adata, n_modules = n_modules)


In [9]:
mod.train()


/gpfs/home/jw7078/.conda/envs/cell2fate_env/lib/python3.9/site-packages/pytorch_lightning/core/lightning.py:2054: DeprecationWarning: `torch.distributed._sharded_tensor` will be deprecated, use `torch.distributed._shard.sharded_tensor` instead
  from torch.distributed._sharded_tensor import pre_load_state_dict_hook, state_dict_hook
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Set SLURM handle signals.


Epoch 500/500: 100%|██████████| 500/500 [19:55<00:00,  2.39s/it, v_num=1, elbo_train=1.77e+6]  


In [10]:
adata = mod.export_posterior(adata,sample_kwargs={'batch_size': None, 'num_samples': 30, 'return_samples': True, 'use_gpu': False})


Sampling global variables, sample: 100%|██████████| 29/29 [00:01<00:00, 15.57it/s]


In [11]:
adata

AnnData object with n_obs × n_vars = 1146 × 395
    obs: 'phase', 'fucci_time', 'pseudo_clusters', 'pseudo_clusters_equal_size', 'pseudo_clusters_equal_size_num', 'cell_cycle_rad', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'n_counts', 'clusters', '_indices', '_scvi_batch', 'Time (hours)', 'Time Uncertainty (sd)'
    var: 'gene_count_corr', 'means', 'dispersions', 'dispersions_norm', 'highly_variable', 'velocity_gamma', 'velocity_qreg_ratio', 'velocity_r2', 'velocity_genes'
    uns: 'neighbors', 'pca', 'umap', 'velocity_params', '_scvi_uuid', '_scvi_manager_uuid', 'mod'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'Ms', 'Mu', 'spliced', 'spliced_raw', 'unspliced', 'unspliced_raw', 'velocity'
    obsp: 'connectivities', 'distances'

In [13]:
adata.write('cell2fate_filteredgene.h5ad')